# 测试不同的 torch.nn.Module 模型复制

In [ ]:
from copy import deepcopy

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchvision
from torchvision.transforms import transforms

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 64

trainset = torchvision.datasets.CIFAR10(root='./CIFAR10/raw', train=True,
                                        download=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=4)

testset = torchvision.datasets.CIFAR10(root='./CIFAR10/raw', train=False,
                                       download=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=4)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
# 初始模型定义
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

## copy.deepcopy() 

In [ ]:
# 模型复制
net_copy = deepcopy(net)
criterion_copy = nn.CrossEntropyLoss()
optimizer_copy = optim.SGD(net_copy.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# 比较
print("是否指向相同内存: {}".format(net is net_copy))

print("比较参数是否相同: {}".format(net.state_dict() == net_copy.state_dict()))

print("比较模型梯度是否相同: {}".format(net.conv1.weight.grad == net_copy.conv1.weight.grad))

In [ ]:
# 训练复制模型
for epoch in range(2):  # loop over the dataset multiple times
    net_copy.train()
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # forward + backward + optimize
        outputs = net_copy(inputs)
        loss = criterion_copy(outputs, labels)
        
        # zero the parameter gradients
        optimizer_copy.zero_grad()
        loss.backward()
        optimizer_copy.step()

In [ ]:
# 再次比较
print("是否指向相同内存: {}".format(net is net_copy))

print("比较参数是否相同: {}".format(net.state_dict() == net_copy.state_dict()))

print("比较模型梯度是否相同: {}".format(net.conv1.weight.grad == net_copy.conv1.weight.grad))